# arXiv Analyzing
### Purpose: 
1. Discover trending research topics
2. Discover how papers are related/build on each other (evolution of a topic)
3. Discover open questions/things to research/missing pieces of research puzzle
4. For fun!

### Mechanics:
Based on Tim Head's code on "Analysing the arXiv" http://betatim.github.io/posts/analysing-the-arxiv/


In [315]:
# Import stuff
import re
import time
import urllib.request
import requests
import urllib
import datetime
import feedparser
feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'
import xml.etree.ElementTree as ET

import pandas as pd
pd.set_option('mode.chained_assignment','warn')
import numpy as np

import matplotlib.pylab as plt

In [316]:
# Build up dataframe of paper titles, id's, abstracts, created dates, cateogry, and doi

OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"

#Harvest from quantum physics
def harvest(beginDate,endDate):

    arxiv="physics:quant-ph"
    df = pd.DataFrame(columns=("title", "abstract", "authors", "categories", "created", "id", "doi"))
    base_url = "http://export.arxiv.org/oai2?verb=ListRecords&"
    url = (base_url +
           "from="+beginDate+"&until="+endDate+"&" +
           "metadataPrefix=arXiv&set=%s"%arxiv)

    while True:
        print("fetching", url)
        try:
            response = urllib.request.urlopen(url).read()

        except urllib.error.HTTPError as e:
            if e.code == 503:
                to = int(e.hdrs.get("retry-after", 30))
                print("Got 503. Retrying after {0:d} seconds.".format(to))

                time.sleep(to)
                continue

            else:
                raise

        root = ET.fromstring(response)

        for record in root.find(OAI+'ListRecords').findall(OAI+"record"):
            authors = ''
            arxiv_id = record.find(OAI+'header').find(OAI+'identifier')
            meta = record.find(OAI+'metadata')
            info = meta.find(ARXIV+"arXiv")
            created = info.find(ARXIV+"created").text
            created = datetime.datetime.strptime(created, "%Y-%m-%d")
            categories = info.find(ARXIV+"categories").text
            try:
                for names in info[3]:
                    authors += (names[1].text+' '+names[0].text)+', '
            except:
                authors += ''


            # if there is more than one DOI use the first one
            # often the second one (if it exists at all) refers
            # to an eratum or similar
            doi = info.find(ARXIV+"doi")
            if doi is not None:
                doi = doi.text.split()[0]

            contents = {'title': info.find(ARXIV+"title").text,
                        'id': info.find(ARXIV+"id").text,#arxiv_id.text[4:],
                        'abstract': info.find(ARXIV+"abstract").text.strip(),
                        'authors': authors,
                        'created': created,
                        'categories': categories.split(),
                        'doi': doi,
                        }

            df = df.append(contents, ignore_index=True)

        # The list of articles returned by the API comes in chunks of
        # 1000 articles. The presence of a resumptionToken tells us that
        # there is more to be fetched.
        token = root.find(OAI+'ListRecords').find(OAI+"resumptionToken")
        if token is None or token.text is None:
            break

        else:
            url = base_url + "resumptionToken=%s"%(token.text)
            
    return df

### Import from arXiv
Import all articles from quant-ph over date range

In [317]:
startDate = '2019-01-01'
endDate = datetime.date.today().__str__()

# Import dataset
df = harvest(startDate, endDate)

fetching http://export.arxiv.org/oai2?verb=ListRecords&from=2019-01-01&until=2020-07-23&metadataPrefix=arXiv&set=physics:quant-ph
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|1001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|2001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|3001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|4001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|5001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|6001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|7001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|8001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|9001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=4739217|10001
fetching http://export.arxiv.

In [318]:
df.head()

,title,abstract,authors,categories,created,id,doi
0,Geometric Phase and Superconducting Flux Quant...,In a ring of s-wave superconducting material t...,,[quant-ph],2007-04-05,0704.0803,None
1,A remark on helical waveguides,Motivated by a proposal to create an optical h...,,"[quant-ph, cond-mat.mes-hall, math-ph, math.MP]",2007-04-20,0704.2770,10.1016/j.physleta.2007.05.013
2,Nontrivial edge coupling from a Dirichlet netw...,In distinction to the Neumann case the squeezi...,,"[math-ph, cond-mat.mes-hall, math.MP, math.SP,...",2007-04-23,0704.2912,10.1088/1751-8113/40/26/F02
3,On the dense point and absolutely continuous s...,"We consider Schr\""odinger operator in dimensio...",,"[math-ph, cond-mat.mes-hall, math.MP, math.SP,...",2007-05-10,0705.1407,10.1007/s11005-007-0191-x
4,"A ""hybrid plane"" with spin-orbit interaction",In this paper we attempt to reconstruct one of...,,"[math-ph, cond-mat.mes-hall, math.MP, math.SP,...",2007-05-17,0705.2487,10.1134/S1061920807040085


In [319]:
# Save DataFrame
df.to_pickle('./arxiv_articles.pkl')

### Import from Saved

In [295]:
# Load DataFrame
df = pd.read_pickle('./arxiv_articles.pkl')

df.tail()

,title,abstract,authors,categories,created,id,doi
18134,Locality Is An Unnecessary Assumption of Bell'...,"Without imposing the locality condition,it is ...",[],[quant-ph],1998-12-14,quant-ph/9812029,None
18135,Quantum Computation with Quantum Dots and Tera...,A quantum computer is proposed in which inform...,"[Mark S. Sherwin, Atac Imamoglu, Thomas Montroy]",[quant-ph],1999-03-17,quant-ph/9903065,10.1103/PhysRevA.60.3508
18136,Enhancement of Magneto-Optic Effects via Large...,We utilize the generation of large atomic cohe...,[],[quant-ph],1999-04-07,quant-ph/9904032,10.1103/PhysRevA.62.023810
18137,Locally curved quantum layers,We consider a quantum particle constrained to ...,[],"[quant-ph, cond-mat, math-ph, math.MP]",1999-10-08,quant-ph/9910035,None
18138,"Large Numbers, the Chinese Remainder Theorem, ...",This is a pedagogical article cited in the for...,[],[quant-ph],1999-11-11,quant-ph/9911051,10.1103/PhysRevApplied.13.024016


### Filter Articles
Pick interested articles based on keywords or authors.

In [28]:
wanted_authors = ['Monroe', 'Richerme', 'Hayes']
keywords = ['trapped ion', 'trapped ions', 'ion trap', 'ion traps']

test = "Sherwin"

np.sum(df['abstract'] == keywords[0])

0

In [312]:
df['authors'].str.contains(r'.*S.*')

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
18134   NaN
18135   NaN
18136   NaN
18137   NaN
18138   NaN
Name: authors, Length: 18139, dtype: float64